# Connect Neo4J to NetworkX
The following gives you a connection to your neo4J DB including the "Network of Thrones" data set. Please make sure everything is set up as described in the main README.

In [1]:
from neo4j import GraphDatabase
import networkx as nx
import os


# Replace with your credentials. Default credentials from main README.
uri = "bolt://localhost:7687"
username = "neo4j"
password = os.environ.get("NEO4J_PASSWORD","password")

with GraphDatabase.driver(uri, auth=(username, password)) as driver:
    driver.verify_connectivity()

def run_query(query, parameters=None):
        """ Run a Cypher query with optional parameters. """
        with driver.session() as session:
            result = session.run(query, parameters)
            return result.data()  # Return query results as a list of dictionaries

This gives you the overall count of movie characters included in all of the 5 books.

In [2]:
query = "MATCH (c:Character) RETURN count(c) AS count"
result = run_query(query)
count = result[0]["count"] if result else 0
print(f"There are {count} characters in the database.")

There are 796 characters in the database.


/var/folders/6j/_bv4mdq511b51s7t6wm__pyr0000gn/T/ipykernel_40451/4153243238.py:16: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


## Task 1.1: Count Interactions

Could you create a query that counts the total number of interactions and stores it in the variable interaction_count

In [8]:
query = "MATCH ()-[r:INTERACTS1|INTERACTS2|INTERACTS3|INTERACTS45]->() RETURN count(r) AS interaction_count"
result = run_query(query)
interaction_count = result[0]["interaction_count"] if result else 0
print(f"There are {interaction_count} interactions in the database.")

There are 3796 interactions in the database.


/var/folders/6j/_bv4mdq511b51s7t6wm__pyr0000gn/T/ipykernel_40451/4153243238.py:16: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


## Task 1.2: Count Interactions by Book

Can you come up with a query that counts the interactions ordered by book?

In [12]:
query = "MATCH ()-[r]->() RETURN r.book AS book, count(r) AS interaction_count ORDER BY interaction_count DESC"
result = run_query(query)

for book in result:
    print(f"Book {book['book']} has {book['interaction_count']} interactions.")

Book 45 has 1329 interactions.
Book 3 has 1008 interactions.
Book 2 has 775 interactions.
Book 1 has 684 interactions.


/var/folders/6j/_bv4mdq511b51s7t6wm__pyr0000gn/T/ipykernel_40451/4153243238.py:16: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:


## Task 1.3: Load the entire Graph to NetworkX

Please complete the missing code in the following cell to load the entire graph into a networkX graph.

In [22]:
query = """
        MATCH (n)
        WITH collect(n) AS nodes
        MATCH ()-[r]->()
        RETURN nodes, collect(r) AS relationships
        """

# Run the query    
result = run_query(query)

# Create a new NetworkX graph
graph = nx.Graph()

# Add nodes to the graph
for record in result:
   for node in record["nodes"]:
      graph.add_node(node["name"])

   # Add relationships to the graph
   for rel in record["relationships"]:
      start_node = rel[0]['name']
      end_node = rel[2]['name']
      graph.add_edge(start_node, end_node)
# Print the graph's nodes and edges
num_edges = len(graph.edges)
num_nodes = len(graph.nodes)
print(f"The graph has {num_nodes} nodes and {num_edges} edges")

The graph has 796 nodes and 2823 edges


/var/folders/6j/_bv4mdq511b51s7t6wm__pyr0000gn/T/ipykernel_40451/4153243238.py:16: DeprecationWarning: Using a driver after it has been closed is deprecated. Future versions of the driver will raise an error.
  with driver.session() as session:
